## Setup

In [ ]:
import reproducibility

In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import logging

# Configuring the number of threads
#NUM_THREADS = 5
#tf.config.threading.set_intra_op_parallelism_threads(NUM_THREADS)
#tf.config.threading.set_inter_op_parallelism_threads(NUM_THREADS)

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
# from keras.utils.vis_utils import plot_model
from keras.utils import plot_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
# from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2
%matplotlib inline

In [ ]:
from utils import logging_setup
logging_setup()

In [ ]:
# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan, load_and_fix_data
from data_funcs import raws_data, synthetic_data, plot_data, check_data, rmse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import run_case, run_rnn, create_RNN_2, staircase, train_rnn, rnn_predict

## User options

In [ ]:
# Change directory for data read/write

dict_file='data/testing_dict.pickle' # input path of FMDA dictionaries
output_path='outputs/outputs.json' # RNN output file
reproducibility_file='data/reproducibility_dict.pickle'

In [ ]:
from module_param_sets import param_sets

In [ ]:
param_sets_keys=['0']
# param_sets_keys = param_sets.keys()

In [ ]:
# read test datasets
test_dict=load_and_fix_data(dict_file)
logging.info("testing datasets test_dict.keys():%s",test_dict.keys())

repro_dict=load_and_fix_data(reproducibility_file)
logging.info("reproducibity dataset repro_dict.keys(): %s",repro_dict.keys())

In [ ]:
output={}
logging.info('params_sets_keys=%s',param_sets_keys)

for i in param_sets_keys:
    print('i=',i)
    params=param_sets[i]
    if i == '0':
        # Run reproducbility case
        print('Running reproducibility')
        assert param_sets[i]['purpose'] == 'reproducibility'
        output[i]={'params':params,'cases':{}}
        case = 'case11'
        case_data=repro_dict[case]
        output[i]['cases'][case]=run_case(case_data,params)
        print('*** params',i,'reproducibility case','summary ***')
        print('params=',params)
        print('outputs=',json.dumps(output[i]['cases'][case],indent=4,sort_keys=True))
        print('writing the results to file',output_path)
        json.dump(output,open(output_path,'w'),indent=4,sort_keys=True)
    else:
        #print('params=',params)
        if params['cases'] == 'all':
            params['cases'] = list(test_dict.keys())
            print("expanding 'all' to",params['cases'])
        output[i]={'params':params,'cases':{}}
        cases = params['cases']
        print('cases=',cases)
        for case in cases:
          for initialize in [True,False]:
            # print(json.dumps(params,indent=4,sort_keys=True))
            case_data=test_dict[case]
            print('case=',case,case_data['title'])
            if not 'title' in case_data.keys():
                case_data['title']=case
            if not 'hours' in case_data.keys():
                case_data['hours']=len(case_data['fm'])
                # case_data['h2']=int(20*24) # length of training period
            if params['synthetic'] or 'Synth' not in case_data['title']: 
                params['initialize']=initialize 
                output[i]['cases'][case]={'initialize':{initialize:run_case(case_data,params)}}  # add to results
                print('*** params',i,'case',case,'summary ***')
                print('params=',params)
                print('outputs=',json.dumps(output[i]['cases'][case],indent=4,sort_keys=True))
                print('writing the results to file',output_path)
                json.dump(output,open(output_path,'w'),indent=4,sort_keys=True)
            else:
                print('skipping synthetic case',case,case_data['title'])
        print('cases=',cases)
print(json.dumps(output,indent=4,sort_keys=True))
   

In [ ]:
logging.info('fmda_rnn_rain.ipynb done')